In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import yfinance as yf
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers


In [20]:
data = pd.read_csv('twitter_BTC_Users_3Months_2021.csv')

## Formatando as mensagens

In [21]:
import re
import emoji
import nltk 
words = set(nltk.corpus.words.words())

def cleaner(text):
    text = re.sub("@[A-Za-z0-9]+","",text) #Remove @ sign
    text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text) #Remove http links
    text = " ".join(text.split())
    text = ''.join(c for c in text if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    text = text.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    text = " ".join(w for w in nltk.wordpunct_tokenize(text) \
         if w.lower() in words or not w.isalpha())
    return text
data['text'] = data['text'].map(lambda x: cleaner(x))
data['date'] = pd.to_datetime(data['date']).dt.date

data.head()

,Unnamed: 0,date,id,text,username,followers,favorites,retweets
0,0,2021-10-29,1454105101988253701,Someone just $ 183m off into cold storage !🐳🐳🐳...,BTC_Archive,960082,7580,769
1,1,2021-10-28,1453718525952999437,"personally $ 1 . 091 Billion ( 17 , 732 )",BTC_Archive,960082,2185,190
2,2,2021-10-28,1453717422402248718,": I personally 17 , 732 $ with $ 9 , 882 avera...",BTC_Archive,960082,6554,557
3,3,2021-10-28,1453689564586037251,Only they seem to apply to .,BTC_Archive,960082,6,0
4,4,2021-10-28,1453673450229354497,there is no hate . It ' s just not comparable ...,BTC_Archive,960082,7,0


## Análise vader

In [22]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

scores = []
# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
for i in range(data['text'].shape[0]):
#print(analyser.polarity_scores(sentiments_pd['text'][i]))
    compound = analyzer.polarity_scores(data['text'][i])["compound"]
    pos = analyzer.polarity_scores(data['text'][i])["pos"]
    neu = analyzer.polarity_scores(data['text'][i])["neu"]
    neg = analyzer.polarity_scores(data['text'][i])["neg"]
    
    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  }) 
sentiments_score = pd.DataFrame.from_dict(scores)
df_sentiment = data.join(sentiments_score)

df_sentiment['Score Positive Retweet'] = (df_sentiment['retweets'] + df_sentiment['favorites']) * df_sentiment['Positive']
df_sentiment['Score Negative Retweet'] = (df_sentiment['retweets'] + df_sentiment['favorites']) * df_sentiment['Negative']
df_sentiment['Score Neutral Retweet'] = (df_sentiment['retweets'] + df_sentiment['favorites']) * df_sentiment['Neutral']

## Removendo colunas desnecessárias e mudando outras

In [23]:
df_sentiment.index = range(df_sentiment.shape[0])
df_sentiment.columns.name = 'Id'
df_sentiment.drop(columns =['username', 'followers', 'Unnamed: 0', 'id'], inplace = True)



data = df_sentiment.groupby(['date']).agg(['mean','count'])
data.columns = [ ' '.join(str(i) for i in col) for col in data.columns]
data.reset_index(inplace=True)
data = data[:-1]
data.drop(columns = 
         ['favorites count', 'retweets count', 'Compound count', 'Positive count', 'Negative count', 'Neutral count', 
         'Score Positive Retweet count', 'Score Negative Retweet count', 'Score Neutral Retweet count'], inplace = True)

In [24]:
data

,date,favorites mean,retweets mean,Compound mean,Positive mean,Negative mean,Neutral mean,Score Positive Retweet mean,Score Negative Retweet mean,Score Neutral Retweet mean
0,2021-07-01,233.714286,35.333333,-0.001395,0.023952,0.026571,0.949429,4.973619,9.916571,254.132190
1,2021-07-02,163.846154,21.153846,0.178592,0.060692,0.013308,0.926000,14.974769,2.195769,167.829462
2,2021-07-03,207.900000,29.500000,0.071900,0.083300,0.010400,0.906300,4.014000,5.491200,227.894800
3,2021-07-04,273.750000,41.250000,0.079550,0.019000,0.000000,0.981000,0.171000,0.000000,314.829000
4,2021-07-05,255.650000,48.100000,0.191020,0.085850,0.044100,0.870100,23.523450,21.401850,259.014900
...,...,...,...,...,...,...,...,...,...,...
116,2021-10-25,127.000000,20.481481,0.064531,0.038833,0.027481,0.933630,2.355926,1.938296,143.182852
117,2021-10-26,169.714286,21.452381,-0.031840,0.034548,0.032833,0.932667,22.493643,1.318071,167.355357
118,2021-10-27,263.517241,33.068966,-0.036252,0.021552,0.034483,0.944034,1.644621,37.841379,257.118172
119,2021-10-28,313.937500,46.312500,0.030342,0.031021,0.025354,0.943625,4.582375,0.537896,355.129729


## Acessando BTC

In [41]:
moeda = ['BTC-USD']
btc = yf.download(moeda, start='2021-07-01',
                end='2021-10-30')


btc ['Return'] = btc['Close'].pct_change()*100
btc ['price dir'] = btc['Close'].diff()
btc ['movement'] =  np.where(btc['price dir'] > 0, 1,0)


#btc = btc[1:]
btc.index = range(btc.shape[0])
btc.columns.name = 'Id'
btc_movement = btc.copy()
btc.drop(columns =['movement'], inplace = True)

btc_movement = btc_movement[1:]
btc_movement.index = range(btc_movement.shape[0])
btc_movement.columns.name = 'Id'
btc_movement.drop(columns =['Open', 'High','Low', 'Close', 'Adj Close', 'Volume', 'Return', 'price dir'], inplace = True)


btc = btc.join(btc_movement)
btc = btc.replace(np.nan,0)
btc_movement


[*********************100%***********************]  1 of 1 completed


Id,movement
0,1
1,1
2,1
3,0
4,1
...,...
116,0
117,0
118,1
119,1


In [42]:
import pandas_ta as ta

#simple movement average
btc['sma5'] = btc.ta.sma(length=5)
btc['sma10'] = btc.ta.sma(length=10)
btc['sma15'] = btc.ta.sma(length=15)

#exponencial movement average
btc['ema5'] = btc.ta.ema(length=5)
btc['ema10'] = btc.ta.ema(length=10)
btc['ema15'] = btc.ta.ema(length=15)

btc

Id,Open,High,Low,Close,Adj Close,Volume,Return,price dir,movement,sma5,sma10,sma15,ema5,ema10,ema15
0,35035.984375,35035.984375,32883.781250,33572.117188,33572.117188,37838957079,0.000000,0.000000,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,33549.601562,33939.589844,32770.679688,33897.046875,33897.046875,38728974942,0.967856,324.929688,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,33854.421875,34909.261719,33402.695312,34668.546875,34668.546875,24383958643,2.276009,771.500000,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,34665.566406,35937.566406,34396.476562,35287.781250,35287.781250,24924307911,1.786156,619.234375,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,35284.343750,35284.343750,33213.660156,33746.003906,33746.003906,26721554282,-4.369154,-1541.777344,1.0,34234.299219,NaN,NaN,34234.299219,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,63032.761719,63229.027344,59991.160156,60363.792969,60363.792969,34878965587,-4.244985,-2676.031250,0.0,61284.067187,62246.503125,61047.655990,61497.424873,61190.595303,60161.459408
118,60352.000000,61435.183594,58208.187500,58482.386719,58482.386719,43657076893,-3.116779,-1881.406250,1.0,60842.091406,61939.380078,61210.411198,60492.412155,60698.193743,59951.575322
119,58470.730469,62128.632812,58206.917969,60622.136719,60622.136719,45257083247,3.658794,2139.750000,1.0,60687.795312,61798.985938,61425.147135,60535.653676,60684.365193,60035.395496
120,60624.871094,62927.609375,60329.964844,62227.964844,62227.964844,36856881767,2.648914,1605.828125,0.0,60947.221094,61595.583203,61752.243229,61099.757399,60965.019675,60309.466665


In [43]:
btc['std 5'] = btc['Close'].rolling(5).std()
btc['std 10'] = btc['Close'].rolling(10).std()
btc['std 15'] = btc['Close'].rolling(15).std()

#proporção candle em relação ao range do dia
btc['prop'] = (btc['Close']-btc['Open'])/(btc['High']-btc['Low'])

#movimento dias anteriores
btc['move -1'] = btc['movement'].shift(1)
btc['move -2'] = btc['movement'].shift(2)
btc['move -3'] = btc['movement'].shift(3)

#Media móvel
btc['MM 5'] = btc['Close'].rolling(5).mean()
btc['MM 10'] = btc['Close'].rolling(10).mean()
btc['MM 15'] = btc['Close'].rolling(15).mean()



btc

Id,Open,High,Low,Close,Adj Close,Volume,Return,price dir,movement,sma5,...,std 5,std 10,std 15,prop,move -1,move -2,move -3,MM 5,MM 10,MM 15
0,35035.984375,35035.984375,32883.781250,33572.117188,33572.117188,37838957079,0.000000,0.000000,1.0,NaN,...,NaN,NaN,NaN,-0.680171,NaN,NaN,NaN,NaN,NaN,NaN
1,33549.601562,33939.589844,32770.679688,33897.046875,33897.046875,38728974942,0.967856,324.929688,1.0,NaN,...,NaN,NaN,NaN,0.297239,1.0,NaN,NaN,NaN,NaN,NaN
2,33854.421875,34909.261719,33402.695312,34668.546875,34668.546875,24383958643,2.276009,771.500000,1.0,NaN,...,NaN,NaN,NaN,0.540384,1.0,1.0,NaN,NaN,NaN,NaN
3,34665.566406,35937.566406,34396.476562,35287.781250,35287.781250,24924307911,1.786156,619.234375,0.0,NaN,...,NaN,NaN,NaN,0.403750,1.0,1.0,1.0,NaN,NaN,NaN
4,35284.343750,35284.343750,33213.660156,33746.003906,33746.003906,26721554282,-4.369154,-1541.777344,1.0,34234.299219,...,722.677645,NaN,NaN,-0.742914,0.0,1.0,1.0,34234.299219,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,63032.761719,63229.027344,59991.160156,60363.792969,60363.792969,34878965587,-4.244985,-2676.031250,0.0,61284.067187,...,1050.784231,1753.574395,2599.692717,-0.824298,0.0,1.0,0.0,61284.067187,62246.503125,61047.655990
118,60352.000000,61435.183594,58208.187500,58482.386719,58482.386719,43657076893,-3.116779,-1881.406250,1.0,60842.091406,...,1653.712596,2119.235821,2325.862070,-0.579366,0.0,0.0,1.0,60842.091406,61939.380078,61210.411198
119,58470.730469,62128.632812,58206.917969,60622.136719,60622.136719,45257083247,3.658794,2139.750000,1.0,60687.795312,...,1625.132637,2158.985166,2085.295552,0.548588,1.0,0.0,0.0,60687.795312,61798.985938,61425.147135
120,60624.871094,62927.609375,60329.964844,62227.964844,62227.964844,36856881767,2.648914,1605.828125,0.0,60947.221094,...,1770.647418,1990.389379,1754.144410,0.617134,1.0,1.0,0.0,60947.221094,61595.583203,61752.243229


In [44]:

df_min_max_scaled = btc.copy() 
for column in df_min_max_scaled.columns: 
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())     
print(df_min_max_scaled)

btc = df_min_max_scaled

Id       Open      High       Low     Close  Adj Close    Volume    Return  \
0    0.144719  0.112175  0.102857  0.104041   0.104041  0.410386  0.528253   
1    0.103666  0.081655  0.099555  0.113020   0.113020  0.429558  0.574471   
2    0.112085  0.108647  0.118008  0.134341   0.134341  0.120545  0.636939   
3    0.134488  0.137272  0.147024  0.151454   0.151454  0.132185  0.613547   
4    0.151579  0.119088  0.112489  0.108846   0.108846  0.170901  0.319612   
..        ...       ...       ...       ...        ...       ...       ...   
117  0.917984  0.896967  0.894320  0.844439   0.844439  0.346623  0.325541   
118  0.843942  0.847033  0.842262  0.792446   0.792446  0.535717  0.379417   
119  0.791982  0.866336  0.842225  0.851579   0.851579  0.570183  0.702972   
120  0.851478  0.888577  0.904212  0.895956   0.895956  0.389231  0.654747   
121  0.896070  0.871946  0.921392  0.886584   0.886584  0.288009  0.502228   

Id   price dir  movement      sma5  ...     std 5    std 10    

## Juntando btc com twitter

In [45]:
df_btc_tweets = data.join(btc)


In [47]:
df_btc_tweets

df_btc_tweets = df_btc_tweets.drop([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,29], axis = 0)

df_btc_tweets

,date,favorites mean,retweets mean,Compound mean,Positive mean,Negative mean,Neutral mean,Score Positive Retweet mean,Score Negative Retweet mean,Score Neutral Retweet mean,...,std 5,std 10,std 15,prop,move -1,move -2,move -3,MM 5,MM 10,MM 15
30,2021-07-31,176.307692,20.000000,0.010946,0.027462,0.031769,0.940769,0.131846,0.158846,196.017000,...,0.276116,0.697767,0.728178,0.223154,0.0,1.0,1.0,0.298108,0.193020,0.107532
31,2021-08-01,404.103448,47.241379,0.001283,0.010069,0.011138,0.978793,7.696828,30.319862,413.328138,...,0.233083,0.585153,0.727923,0.230263,0.0,0.0,1.0,0.301643,0.218067,0.126636
32,2021-08-02,441.606061,52.909091,-0.039191,0.020788,0.029212,0.950061,12.636242,21.415061,460.473182,...,0.295754,0.466272,0.706467,0.268858,0.0,0.0,0.0,0.296701,0.236441,0.143394
33,2021-08-03,190.571429,24.607143,0.011696,0.019714,0.018643,0.961607,6.666500,10.383500,198.125107,...,0.439269,0.337299,0.641680,0.216697,0.0,0.0,0.0,0.285153,0.249062,0.159994
34,2021-08-04,429.294118,61.411765,0.040906,0.025471,0.005353,0.969176,63.773412,0.236235,426.696235,...,0.295693,0.203823,0.531892,0.830659,1.0,0.0,0.0,0.269667,0.263438,0.182489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2021-10-25,127.000000,20.481481,0.064531,0.038833,0.027481,0.933630,2.355926,1.938296,143.182852,...,0.193790,0.264478,0.406929,0.860696,1.0,0.0,1.0,0.951590,1.000000,0.968900
117,2021-10-26,169.714286,21.452381,-0.031840,0.034548,0.032833,0.932667,22.493643,1.318071,167.355357,...,0.222086,0.277653,0.373419,0.057604,0.0,1.0,0.0,0.940098,0.998273,0.975415
118,2021-10-27,263.517241,33.068966,-0.036252,0.021552,0.034483,0.944034,1.644621,37.841379,257.118172,...,0.427348,0.363730,0.314489,0.186104,0.0,0.0,1.0,0.926344,0.988232,0.980940
119,2021-10-28,313.937500,46.312500,0.030342,0.031021,0.025354,0.943625,4.582375,0.537896,355.129729,...,0.417618,0.373088,0.262718,0.777869,1.0,0.0,0.0,0.921542,0.983643,0.988229


## Criando regressor

In [83]:
regressor = Sequential()

regressor.add(Dense(150, input_dim=33, kernel_initializer='random_uniform',
                    activation='sigmoid', use_bias=False))
regressor.add(Dense(200, kernel_initializer='random_uniform', 
                    activation='sigmoid', use_bias=False))

regressor.add(Dense(1, kernel_initializer='random_uniform',
                    activation='linear', use_bias=False))

regressor.compile(loss='mean_squared_error', optimizer='adam')

regressor.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 150)               4950      
                                                                 
 dense_4 (Dense)             (None, 200)               30000     
                                                                 
 dense_5 (Dense)             (None, 1)                 200       
                                                                 
Total params: 35,150
Trainable params: 35,150
Non-trainable params: 0
_________________________________________________________________


## Escalando dados

In [52]:
df_btc_tweets.columns

Index(['date', 'favorites mean', 'retweets mean', 'Compound mean',
       'Positive mean', 'Negative mean', 'Neutral mean',
       'Score Positive Retweet mean', 'Score Negative Retweet mean',
       'Score Neutral Retweet mean', 'Open', 'High', 'Low', 'Close',
       'Adj Close', 'Volume', 'Return', 'price dir', 'movement', 'sma5',
       'sma10', 'sma15', 'ema5', 'ema10', 'ema15', 'std 5', 'std 10', 'std 15',
       'prop', 'move -1', 'move -2', 'move -3', 'MM 5', 'MM 10', 'MM 15'],
      dtype='object')

In [58]:
#important_columns = ["followers mean", "favorites mean", "retweets mean", "Compound mean", "Positive mean", "Negative mean", "Neutral mean", 'Score Positive Retweet mean', 'Score Negative Retweet mean','Score Neutral Retweet mean','std 5','std 15','prop','move -1','move -2','MM 5','MM 10','sma10', 'ema5',]
important_columns = ['date', 'favorites mean', 'retweets mean', 'Compound mean',
       'Positive mean', 'Negative mean', 'Neutral mean',
       'Score Positive Retweet mean', 'Score Negative Retweet mean',
       'Score Neutral Retweet mean', 'Open', 'High', 'Low', 'Close',
       'Adj Close', 'Volume', 'Return', 'price dir', 'sma5',
       'sma10', 'sma15', 'ema5', 'ema10', 'ema15', 'std 5', 'std 10', 'std 15',
       'prop', 'move -1', 'move -2', 'move -3', 'MM 5', 'MM 10', 'MM 15']

x = df_btc_tweets[important_columns]
y = df_btc_tweets[["movement"]]
y = y.astype(int)



In [62]:
#import imblearn
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# usar técnica under-sampling
rus = RandomOverSampler()
x_res, y_res = rus.fit_resample(x_train, y_train)

### Prox passo, descobrir como associar com exemplo "Loading the numerical and categorical data" do site https://pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

In [59]:
df_btc_tweets.shape

(91, 35)

In [57]:
cols = df_btc_tweets.columns.tolist()
cols

['date',
 'favorites mean',
 'retweets mean',
 'Compound mean',
 'Positive mean',
 'Negative mean',
 'Neutral mean',
 'Score Positive Retweet mean',
 'Score Negative Retweet mean',
 'Score Neutral Retweet mean',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'Return',
 'price dir',
 'movement',
 'sma5',
 'sma10',
 'sma15',
 'ema5',
 'ema10',
 'ema15',
 'std 5',
 'std 10',
 'std 15',
 'prop',
 'move -1',
 'move -2',
 'move -3',
 'MM 5',
 'MM 10',
 'MM 15']

In [ ]:
important_columns = ['favorites mean',
 'favorites count',
 'retweets mean',
 'retweets count',
 'Compound mean',
 'Compound count',
 'Positive mean',
 'Positive count',
 'Negative mean',
 'Negative count',
 'Neutral mean',
 'Neutral count']


x = df_btc_tweets[important_columns]
y = df_btc_tweets[['movement']]

In [81]:


x_train.columns.tolist()
x_train = x[:85]
y_train = y[:85]
x_test = x[85:]
y_test = y[85:]

x_train.drop(['date'], axis=1, inplace=True)

x_test.drop(['date'], axis=1, inplace=True)
#x_train = np.asarray(x_train).astype(np.float32)
#y_train = np.asarray(y_train).astype(np.float32)


x_train.columns.tolist()

['favorites mean',
 'retweets mean',
 'Compound mean',
 'Positive mean',
 'Negative mean',
 'Neutral mean',
 'Score Positive Retweet mean',
 'Score Negative Retweet mean',
 'Score Neutral Retweet mean',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'Return',
 'price dir',
 'sma5',
 'sma10',
 'sma15',
 'ema5',
 'ema10',
 'ema15',
 'std 5',
 'std 10',
 'std 15',
 'prop',
 'move -1',
 'move -2',
 'move -3',
 'MM 5',
 'MM 10',
 'MM 15']

In [84]:
regressor.fit(x_train, y_train, epochs=1024, shuffle=True, batch_size = 128, validation_data = (x_test, y_test))

Epoch 1/1024
1/1 [==============================] - 2s 2s/step - loss: 0.7777 - val_loss: 0.4687
Epoch 2/1024
1/1 [==============================] - 0s 40ms/step - loss: 0.5136 - val_loss: 0.3135
Epoch 3/1024
1/1 [==============================] - 0s 49ms/step - loss: 0.3415 - val_loss: 0.2484
Epoch 4/1024
1/1 [==============================] - 0s 43ms/step - loss: 0.2583 - val_loss: 0.2590
Epoch 5/1024
1/1 [==============================] - 0s 47ms/step - loss: 0.2509 - val_loss: 0.3133
Epoch 6/1024
1/1 [==============================] - 0s 48ms/step - loss: 0.2897 - val_loss: 0.3686
Epoch 7/1024
1/1 [==============================] - 0s 48ms/step - loss: 0.3347 - val_loss: 0.3975
Epoch 8/1024
1/1 [==============================] - 0s 50ms/step - loss: 0.3588 - val_loss: 0.3954
Epoch 9/1024
1/1 [==============================] - 0s 50ms/step - loss: 0.3565 - val_loss: 0.3702
Epoch 10/1024
1/1 [==============================] - 0s 45ms/step - loss: 0.3351 - val_loss: 0.3340
Epoch 11/10

Epoch 83/1024
1/1 [==============================] - 0s 63ms/step - loss: 0.2360 - val_loss: 0.2222
Epoch 84/1024
1/1 [==============================] - 0s 53ms/step - loss: 0.2358 - val_loss: 0.2219
Epoch 85/1024
1/1 [==============================] - 0s 48ms/step - loss: 0.2357 - val_loss: 0.2217
Epoch 86/1024
1/1 [==============================] - 0s 51ms/step - loss: 0.2355 - val_loss: 0.2214
Epoch 87/1024
1/1 [==============================] - 0s 68ms/step - loss: 0.2354 - val_loss: 0.2211
Epoch 88/1024
1/1 [==============================] - 0s 118ms/step - loss: 0.2352 - val_loss: 0.2207
Epoch 89/1024
1/1 [==============================] - 0s 160ms/step - loss: 0.2350 - val_loss: 0.2201
Epoch 90/1024
1/1 [==============================] - 0s 89ms/step - loss: 0.2348 - val_loss: 0.2195
Epoch 91/1024
1/1 [==============================] - 0s 67ms/step - loss: 0.2347 - val_loss: 0.2188
Epoch 92/1024
1/1 [==============================] - 0s 52ms/step - loss: 0.2345 - val_loss: 0.218

1/1 [==============================] - 0s 56ms/step - loss: 0.2200 - val_loss: 0.1883
Epoch 165/1024
1/1 [==============================] - 0s 62ms/step - loss: 0.2197 - val_loss: 0.1879
Epoch 166/1024
1/1 [==============================] - 0s 54ms/step - loss: 0.2194 - val_loss: 0.1876
Epoch 167/1024
1/1 [==============================] - 0s 49ms/step - loss: 0.2192 - val_loss: 0.1873
Epoch 168/1024
1/1 [==============================] - 0s 50ms/step - loss: 0.2189 - val_loss: 0.1870
Epoch 169/1024
1/1 [==============================] - 0s 54ms/step - loss: 0.2186 - val_loss: 0.1867
Epoch 170/1024
1/1 [==============================] - 0s 41ms/step - loss: 0.2184 - val_loss: 0.1864
Epoch 171/1024
1/1 [==============================] - 0s 45ms/step - loss: 0.2181 - val_loss: 0.1861
Epoch 172/1024
1/1 [==============================] - 0s 47ms/step - loss: 0.2178 - val_loss: 0.1858
Epoch 173/1024
1/1 [==============================] - 0s 68ms/step - loss: 0.2175 - val_loss: 0.1855
Epoch

1/1 [==============================] - 0s 49ms/step - loss: 0.1907 - val_loss: 0.1507
Epoch 246/1024
1/1 [==============================] - 0s 57ms/step - loss: 0.1902 - val_loss: 0.1501
Epoch 247/1024
1/1 [==============================] - 0s 53ms/step - loss: 0.1897 - val_loss: 0.1495
Epoch 248/1024
1/1 [==============================] - 0s 52ms/step - loss: 0.1893 - val_loss: 0.1488
Epoch 249/1024
1/1 [==============================] - 0s 78ms/step - loss: 0.1888 - val_loss: 0.1482
Epoch 250/1024
1/1 [==============================] - 0s 298ms/step - loss: 0.1883 - val_loss: 0.1475
Epoch 251/1024
1/1 [==============================] - 0s 142ms/step - loss: 0.1878 - val_loss: 0.1469
Epoch 252/1024
1/1 [==============================] - 0s 114ms/step - loss: 0.1873 - val_loss: 0.1463
Epoch 253/1024
1/1 [==============================] - 0s 110ms/step - loss: 0.1869 - val_loss: 0.1457
Epoch 254/1024
1/1 [==============================] - 0s 138ms/step - loss: 0.1864 - val_loss: 0.1451


1/1 [==============================] - 0s 42ms/step - loss: 0.1513 - val_loss: 0.0884
Epoch 327/1024
1/1 [==============================] - 0s 44ms/step - loss: 0.1509 - val_loss: 0.0876
Epoch 328/1024
1/1 [==============================] - 0s 40ms/step - loss: 0.1504 - val_loss: 0.0865
Epoch 329/1024
1/1 [==============================] - 0s 72ms/step - loss: 0.1500 - val_loss: 0.0861
Epoch 330/1024
1/1 [==============================] - 0s 38ms/step - loss: 0.1496 - val_loss: 0.0847
Epoch 331/1024
1/1 [==============================] - 0s 38ms/step - loss: 0.1492 - val_loss: 0.0854
Epoch 332/1024
1/1 [==============================] - 0s 40ms/step - loss: 0.1489 - val_loss: 0.0829
Epoch 333/1024
1/1 [==============================] - 0s 47ms/step - loss: 0.1485 - val_loss: 0.0838
Epoch 334/1024
1/1 [==============================] - 0s 39ms/step - loss: 0.1479 - val_loss: 0.0817
Epoch 335/1024
1/1 [==============================] - 0s 36ms/step - loss: 0.1473 - val_loss: 0.0807
Epoch

...

1/1 [==============================] - 0s 46ms/step - loss: 0.0602 - val_loss: 0.1407
Epoch 651/1024
1/1 [==============================] - 0s 48ms/step - loss: 0.0577 - val_loss: 0.1315
Epoch 652/1024
1/1 [==============================] - 0s 49ms/step - loss: 0.0569 - val_loss: 0.1081
Epoch 653/1024
1/1 [==============================] - 0s 48ms/step - loss: 0.0580 - val_loss: 0.1592
Epoch 654/1024
1/1 [==============================] - 0s 46ms/step - loss: 0.0591 - val_loss: 0.1038
Epoch 655/1024
1/1 [==============================] - 0s 41ms/step - loss: 0.0586 - val_loss: 0.1478
Epoch 656/1024
1/1 [==============================] - 0s 37ms/step - loss: 0.0570 - val_loss: 0.1275
Epoch 657/1024
1/1 [==============================] - 0s 46ms/step - loss: 0.0559 - val_loss: 0.1187
Epoch 658/1024
1/1 [==============================] - 0s 42ms/step - loss: 0.0561 - val_loss: 0.1532
Epoch 659/1024
1/1 [==============================] - 0s 41ms/step - loss: 0.0568 - val_loss: 0.1096
Epoch

1/1 [==============================] - 0s 47ms/step - loss: 0.0423 - val_loss: 0.1571
Epoch 732/1024
1/1 [==============================] - 0s 50ms/step - loss: 0.0417 - val_loss: 0.1603
Epoch 733/1024
1/1 [==============================] - 0s 36ms/step - loss: 0.0416 - val_loss: 0.1411
Epoch 734/1024
1/1 [==============================] - 0s 36ms/step - loss: 0.0418 - val_loss: 0.1663
Epoch 735/1024
1/1 [==============================] - 0s 48ms/step - loss: 0.0416 - val_loss: 0.1487
Epoch 736/1024
1/1 [==============================] - 0s 41ms/step - loss: 0.0411 - val_loss: 0.1507
Epoch 737/1024
1/1 [==============================] - 0s 39ms/step - loss: 0.0409 - val_loss: 0.1640
Epoch 738/1024
1/1 [==============================] - 0s 45ms/step - loss: 0.0410 - val_loss: 0.1435
Epoch 739/1024
1/1 [==============================] - 0s 52ms/step - loss: 0.0409 - val_loss: 0.1620
Epoch 740/1024
1/1 [==============================] - 0s 55ms/step - loss: 0.0406 - val_loss: 0.1534
Epoch

1/1 [==============================] - 0s 67ms/step - loss: 0.0319 - val_loss: 0.1431
Epoch 813/1024
1/1 [==============================] - 0s 52ms/step - loss: 0.0321 - val_loss: 0.1754
Epoch 814/1024
1/1 [==============================] - 0s 56ms/step - loss: 0.0314 - val_loss: 0.1643
Epoch 815/1024
1/1 [==============================] - 0s 44ms/step - loss: 0.0309 - val_loss: 0.1489
Epoch 816/1024
1/1 [==============================] - 0s 47ms/step - loss: 0.0312 - val_loss: 0.1803
Epoch 817/1024
1/1 [==============================] - 0s 47ms/step - loss: 0.0314 - val_loss: 0.1512
Epoch 818/1024
1/1 [==============================] - 0s 46ms/step - loss: 0.0309 - val_loss: 0.1631
Epoch 819/1024
1/1 [==============================] - 0s 43ms/step - loss: 0.0305 - val_loss: 0.1731
Epoch 820/1024
1/1 [==============================] - 0s 37ms/step - loss: 0.0306 - val_loss: 0.1491
Epoch 821/1024
1/1 [==============================] - 0s 43ms/step - loss: 0.0308 - val_loss: 0.1751
Epoch

1/1 [==============================] - 0s 44ms/step - loss: 0.0238 - val_loss: 0.2011
Epoch 894/1024
1/1 [==============================] - 0s 39ms/step - loss: 0.0259 - val_loss: 0.1294
Epoch 895/1024
1/1 [==============================] - 0s 42ms/step - loss: 0.0263 - val_loss: 0.1827
Epoch 896/1024
1/1 [==============================] - 0s 39ms/step - loss: 0.0241 - val_loss: 0.1763
Epoch 897/1024
1/1 [==============================] - 0s 37ms/step - loss: 0.0237 - val_loss: 0.1353
Epoch 898/1024
1/1 [==============================] - 0s 42ms/step - loss: 0.0251 - val_loss: 0.1954
Epoch 899/1024
1/1 [==============================] - 0s 37ms/step - loss: 0.0249 - val_loss: 0.1538
Epoch 900/1024
1/1 [==============================] - 0s 42ms/step - loss: 0.0233 - val_loss: 0.1500
Epoch 901/1024
1/1 [==============================] - 0s 51ms/step - loss: 0.0234 - val_loss: 0.1906
Epoch 902/1024
1/1 [==============================] - 0s 46ms/step - loss: 0.0244 - val_loss: 0.1399
Epoch

1/1 [==============================] - 0s 47ms/step - loss: 0.0335 - val_loss: 0.2170
Epoch 975/1024
1/1 [==============================] - 0s 40ms/step - loss: 0.0224 - val_loss: 0.1668
Epoch 976/1024
1/1 [==============================] - 0s 42ms/step - loss: 0.0177 - val_loss: 0.1118
Epoch 977/1024
1/1 [==============================] - 0s 49ms/step - loss: 0.0231 - val_loss: 0.2492
Epoch 978/1024
1/1 [==============================] - 0s 41ms/step - loss: 0.0282 - val_loss: 0.1096
Epoch 979/1024
1/1 [==============================] - 0s 35ms/step - loss: 0.0238 - val_loss: 0.1773
Epoch 980/1024
1/1 [==============================] - 0s 43ms/step - loss: 0.0178 - val_loss: 0.1965
Epoch 981/1024
1/1 [==============================] - 0s 38ms/step - loss: 0.0193 - val_loss: 0.1104
Epoch 982/1024
1/1 [==============================] - 0s 45ms/step - loss: 0.0235 - val_loss: 0.2164
Epoch 983/1024
1/1 [==============================] - 0s 54ms/step - loss: 0.0220 - val_loss: 0.1440
Epoch

In [88]:

x = np.asarray(x_train).astype(np.float32)
y = np.asarray(y_train).astype(np.float32)

accuracy = regressor.evaluate(x, y)
print('Accuracy: %.2f' % (accuracy*100))

3/3 [==============================] - 1s 3ms/step - loss: 0.0149
Accuracy: 1.49


In [89]:
y_predict = regressor.predict(x_train)
y_predict_teste = regressor.predict(x_test)

In [90]:
accuracy = regressor.evaluate(x, y)
print('Accuracy: %.2f' % (accuracy*100))

3/3 [==============================] - 0s 3ms/step - loss: 0.0149
Accuracy: 1.49


In [91]:
accuracy = regressor.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

1/1 [==============================] - 0s 39ms/step - loss: 0.1489
Accuracy: 14.89
